In [1]:
!apt install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
make is already the newest version (4.1-9.1ubuntu1).
make set to manually installed.
xz-utils is already the newest version (5.2.2-1.3).
xz-utils set to manually installed.
libmecab-dev is already the newest version (0.996-5).
mecab is already the newest version (0.996-5).
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-1).
curl is already the newest version (7.58.0-2ubuntu3.5).
file is already the newest version (1:5.32-2ubuntu0.1).
git is already the newest version (1:2.17.1-1ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [2]:
!pip install mecab-python3==0.7

    100% |████████████████████████████████| 51kB 3.9MB/s 
  Running setup.py bdist_wheel for mecab-python3 ... - \ | / done
  Stored in directory: /root/.cache/pip/wheels/4c/07/3a/5f22ccc9f381f3bc01fa023202061cd1e0e9af855292f005dd
Successfully built mecab-python3
  Found existing installation: mecab-python3 0.996.1
    Uninstalling mecab-python3-0.996.1:
      Successfully uninstalled mecab-python3-0.996.1


In [0]:
import MeCab
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

text = open('simuraly-data.txt', 'r').read()
documents = text.replace("\n", "").split("end_of_text")

def words(text):
    # 文章を単語に分割する
    word_list = []
    tagger = MeCab.Tagger('-Ochasen')
    tagger.parse('')
    node = tagger.parseToNode(text)

    file = open('words.txt', 'a')
    while node:
        word_type = node.feature.split(",")[0]
        word = node.surface
        if word == '*' and node.feature.split(",")[6].isalpha():
            word = node.feature.split(",")[6]

        if len(word) > 1 and word.isalpha() and word_type in ["名詞"]:
            word_list.append(word.lower())
            file.write(str(word + "\n"))
            
        node = node.next
    
    file.close()
    return word_list

# テキストごとに単語分割して学習データを作成する
train_data = []
file = open('documents.txt', 'w')

for i, document in enumerate(documents):
    train_data.append(TaggedDocument(words=words(document), tags=['text' + str(i + 1)]))
    file.write("*****" + str(words(document)) + "\n")

file.close()
    

# doc2vec の学習を実行
model = Doc2Vec(documents=train_data, min_count=1, alpha=0.005, epochs=40, sample=20, dm=1)



In [93]:
for i, document in enumerate(documents):
    print('text' + str(i + 1) + "\t" + document.replace("\n", "")[0:50])
    for items in model.docvecs.most_similar('text' + str(i + 1), topn=2):
        print("\t" + str(items[0]) + " : \t"+ str(items[1]) + "\t")

text1	マイコンボードを中継器にして PC に電子部品を接続し、Ruby で制御してみた（発光ダイオード編）
	text7 : 	0.9987452030181885	
	text5 : 	0.9986720085144043	
text2	マイコンボードを中継器にして PC に電子部品を接続し、Ruby で制御してみた（温度センサ編）Ru
	text7 : 	0.9979823231697083	
	text1 : 	0.9978688955307007	
text3	フロントエンド初心者が、ReactでHelloWorldしてみた（ブラウザ上編）初心者ブラウザHel
	text21 : 	0.9979885220527649	
	text20 : 	0.9979159235954285	
text4	フロントエンド初心者が、Vue.jsでHelloWorldしてみた（ブラウザ上編）初心者ブラウザHe
	text20 : 	0.9967581033706665	
	text21 : 	0.9964686036109924	
text5	マイコンボードを中継器にして PC に電子部品を接続し、Ruby で制御してみた（サーボモータ編）R
	text1 : 	0.9986720085144043	
	text7 : 	0.9985793232917786	
text6	最初に覚えて欲しいPythonのロギングPython21はじめに簡単な使い捨てのコードならprint
	text19 : 	0.9987303018569946	
	text7 : 	0.9971414804458618	
text7	マイコンボードを中継器にして PC に電子部品を接続し、Ruby で制御してみた（人感センサ編）Ru
	text20 : 	0.9989395141601562	
	text1 : 	0.9987452626228333	
text8	proxy が存在するネットワークから、インターネット上の何かにアクセスするproxy5この記事は、
	text18 : 	0.8662922382354736	
	text16 : 	0.8357861638069153	
text9	Adobe XD CCでモックアップをサクサク作ろうAdobeXDモックアップ1今更ですが、Adob

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [136]:
for result in model.most_similar(positive="コマンド", topn=10):
      print(str(result))

('設定', 0.9952303171157837)
('以下', 0.9942916631698608)
('情報', 0.9937410950660706)
('ファイル', 0.9932984113693237)
('fiddler', 0.9932637214660645)
('作成', 0.99315345287323)
('環境', 0.99250328540802)
('記事', 0.9917477369308472)
('id', 0.9915721416473389)
('必要', 0.9906970262527466)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [137]:
for result in model.most_similar(positive=["コマンド", "user"], topn=10):
      print(str(result))

('https', 0.991931676864624)
('ネットワーク', 0.9820094108581543)
('存在', 0.9805724024772644)
('情報', 0.978228747844696)
('ファイル', 0.977412223815918)
('設定', 0.973896861076355)
('git', 0.9735790491104126)
('fiddler', 0.9706029295921326)
('インターネット', 0.9678129553794861)
('作成', 0.9671698808670044)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [142]:
for result in model.most_similar(positive=["コマンド", "user"], negative=["http"], topn=10):
      print(str(result))

('python', 0.9798836708068848)
('pc', 0.9794877767562866)
('arduino', 0.9788479804992676)
('pip', 0.978778600692749)
('name', 0.9787502288818359)
('self', 0.9783775806427002)
('制御', 0.9783043265342712)
('インストール', 0.9781792759895325)
('keras', 0.9780073165893555)
('logger', 0.9779103994369507)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [4]:
import MeCab

text = "私はバナナが好きです。"

mecabTagger = MeCab.Tagger("mecabrc")
mecabTagger.parse("")
node = mecabTagger.parseToNode(text)
while node:
    word = node.surface
    # hinshi = node.feature.split(",")[0]
    print(word+": "+ node.feature)
    node = node.next

: BOS/EOS,*,*,*,*,*,*,*,*
私: 名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ
は: 助詞,係助詞,*,*,*,*,は,ハ,ワ
バナナ: 名詞,一般,*,*,*,*,バナナ,バナナ,バナナ
が: 助詞,格助詞,一般,*,*,*,が,ガ,ガ
好き: 名詞,形容動詞語幹,*,*,*,*,好き,スキ,スキ
です: 助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
。: 記号,句点,*,*,*,*,。,。,。
: BOS/EOS,*,*,*,*,*,*,*,*


In [44]:
print('*'.isalpha())

False
